In [ ]:
from collections import deque

def solution(n, infection, edges, k):
    # 편의를 위해 0-based 인덱스로 변환
    infection -= 1

    # 간선 타입별 인접 리스트(그래프) 생성
    # table[1] : 타입 1 간선으로만 만든 그래프의 인접 리스트
    # table[2] : 타입 2 ...
    # table[3] : 타입 3 ...
    # 인덱스를 0,1,2 대신 1,2,3으로 쓰기 때문에 크기는 4로 만들고 0은 안 씀
    table = {1:[[] for _ in range(n)], 2:[[] for _ in range(n)], 3:[[] for _ in range(n)]}
    for x, y, t in edges:
        x -= 1; y -= 1  # 정점도 0-based
        table[t][x].append(y)
        table[t][y].append(x)

    # mask[t-1][i] = "간선 타입 t 로만 이동했을 때, 정점 i 가 속한 연결요소(component)의 노드 집합 비트마스크"
    # 예) mask[0][i] 는 타입 1 그래프에서 i 와 같은 컴포넌트에 있는 모든 정점을 비트마스크로 표현
    mask = [[0]*n, [0]*n, [0]*n]

    # 각 간선 타입(t=1,2,3)에 대해 연결요소를 미리 구해 두기
    for t in (1, 2, 3):
        seen = [0]*n  # 해당 타입 그래프에서 방문 여부
        for s in range(n):
            if seen[s]:
                continue
            # BFS(또는 DFS)로 하나의 연결요소(component) 탐색
            q1 = deque([s])
            seen[s] = 1
            comp = [s]  # 현재 컴포넌트에 속한 정점들

            while q1:
                u = q1.popleft()
                for v in table[t][u]:
                    if not seen[v]:
                        seen[v] = 1
                        q1.append(v)
                        comp.append(v)

            # comp 에 있는 노드들을 하나의 비트마스크로 만들기
            bit = 0
            for p in comp:
                bit |= 1 << p

            # comp 에 속한 모든 노드 i 에 대해
            # "i 에서 시작해 타입 t 간선만 타고 갈 수 있는 노드 집합" 이 동일하므로 bit 를 그대로 저장
            for q in comp:
                mask[t-1][q] = bit

# 
    # trans[t] : 상태 비트마스크 S에 대해, 타입 t를 사용했을 때의 결과 상태를 캐싱하는 딕셔너리
    # (동일한 S, t 조합을 여러 번 계산하지 않기 위한 메모이제이션)
    trans = {1:{}, 2:{}, 3:{}}

    def step(S, t):
        """
        현재 감염 상태 S(비트마스크)에서 간선 타입 t만 허용해서.
        '전염을 한 번' 일으켰을 때의 새로운 감염 상태를 계산.
        S에 포함된 각 정점이 속한 (타입 t 기준) 컴포넌트 전체가 감염된다고 보면 됨.
        """
        cache = trans[t]
        if S in cache:
            return cache[S]

        res, s0 = 0, S

        # S에 포함된 모든 정점에 대해:
        # 해당 정점이 속한 컴포넌트 비트마스크(mask[t-1][i])를 OR 로 합쳐서 결과 집합(res)을 만든다.
        while s0:
            lsb = s0 & -s0           # S에서 가장 낮은 비트(LSB)만 추출
            i = lsb.bit_length() - 1 # 해당 비트에 해당하는 정점 번호
            res |= mask[t-1][i]      # 정점 i 의 컴포넌트 전체를 결과에 포함
            s0 ^= lsb                # 처리한 비트 제거

        cache[S] = res
        return res

    # 시작 상태: 초기 감염 정점 하나만 1 로 켜진 비트마스크
    start = 1 << infection
    best = 1  # 지금까지 발견한 최대 감염 노드 수

    # k == 0 이면 전염을 시도할 수 없으므로, 최초 감염 노드 하나만 가능
    if k == 0:
        return 1

    # 상태 공간을 BFS 로 탐색
    # 큐 원소: (현재 감염 상태 비트마스크, 지금까지 사용한 단계 수)
    q2 = deque([(start, 0)])
    # seen_depth[S] = 상태 S 를 최소 몇 단계에서 처음 방문했는지 기록
    seen_depth = {start: 0}

    while q2:
        S, d = q2.popleft()

        # 현재 상태에서 감염된 노드 수
        c = S.bit_count()
        if c > best:
            best = c
            # 모든 노드가 감염되면 더 볼 필요 없이 바로 종료
            if best == n:
                return n

        # 이미 k 번의 전염을 모두 사용했다면, 더 진행 불가
        if d == k:
            continue

        # 다음 단계에서 사용할 간선 타입 t 를 1,2,3 중에서 하나 선택
        for t in (1, 2, 3):
            S2 = step(S, t)  # 타입 t 를 사용했을 때의 새로운 감염 상태

            # 상태가 변하지 않으면(진행 없음) 스킵
            if S2 == S:
                continue

            nd = d + 1  # 한 단계 진행

            # 예전에 S2 를 더 적은(또는 같은) 단계 수로 방문했다면,
            # 지금 다시 올 필요 없음 (더 나은 경로가 아님)
            if seen_depth.get(S2, 1 << 30) <= nd:
                continue

            seen_depth[S2] = nd
            q2.append((S2, nd))

    # k 단계 이내에 도달 가능한 상태들 중에서 감염 노드 수의 최대값 반환
    return best


In [ ]:
def solution(dist_limit: int, split_limit: int) -> int:
    # 문제에서 주어진 파라미터를 짧은 이름으로 치환
    # D : 사용할 수 있는 '분할(거리) 예산' (총 몇 번의 분할/단계를 수행할 수 있는지)
    # S : 한 경로에서 허용되는 최대 분기(= 2,3 분할을 곱한 값의 상한)
    D, S = dist_limit, split_limit

    # 더 이상 분할할 수 없거나, 분기 상한이 1 이하이면
    # 루트 하나뿐인 트리(leaf = 1)만 가능
    if D == 0 or S < 2: 
        return 1

    # --- 2, 3의 거듭제곱 미리 생성 ---
    # p2[i] = 2^i (단, 2^i <= S 인 최대까지)
    p2 = [1]
    while p2[-1] * 2 <= S:
        p2.append(p2[-1] * 2)

    # p3[i] = 3^i (단, 3^i <= S 인 최대까지)
    p3 = [1]
    while p3[-1] * 3 <= S:
        p3.append(p3[-1] * 3)

    # 지금까지 만들 수 있었던 leaf(말단 노드)의 최대 개수
    best = 1

    # b : 2로 분기하는 횟수(= 몇 레벨에서 2분기를 사용할지)
    # v2 = 2^b
    for b, v2 in enumerate(p2):
        # 2^b * 3^a <= S 를 만족하도록 a 의 최대 범위를 구해야 하므로
        # 3^a <= S / 2^b 이 되어야 한다.
        remain = S // v2

        # a : 3으로 분기하는 횟수(= 몇 레벨에서 3분기를 사용할지)
        # p3[a] 가 remain 이하인 최대 a 를 찾는다.
        a = 0
        while a + 1 < len(p3) and p3[a + 1] <= remain:
            a += 1

        # (2,3 분기를 어떤 순서로 할지 두 경우 모두 시도)
        # order = 0 → 2 먼저, 그 다음 3
        # order = 1 → 3 먼저, 그 다음 2
        for order in (0, 1):
            # leaves : 현재 트리의 leaf 개수
            # cap    : 현재 레벨에서 '분할 가능한 노드 수'(frontier 노드 수)
            # rem    : 남은 분할 예산 D
            leaves, cap, rem = 1, 1, D

            # seq = ((분기수 k, 그 레벨 반복 횟수 cnt), ...)
            # 예: order == 0 일 때 2 분기 b번 → 3 분기 a번
            seq = ((2, b), (3, a)) if order == 0 else ((3, a), (2, b))

            # 각 분기 타입(k=2 또는 3)에 대해, cnt 번 레벨을 내려가면서 시뮬레이션
            for k, cnt in seq:
                for _ in range(cnt):
                    # 더 이상 분할 예산이 없으면 종료
                    if rem == 0:
                        break

                    # 이번 레벨에서 실제로 분기 시도할 수 있는 노드 수
                    #   - cap 개까지 분할 가능하지만
                    #   - 분할 하나가 '거리 1'을 소모한다고 보면,
                    #     rem 을 초과해서는 분할할 수 없다.
                    use = cap if cap <= rem else rem

                    # leaf 노드 하나를 k-분기로 나누면
                    #   기존 leaf 1개가 내부 노드로 바뀌고,
                    #   새로 k 개의 자식이 생김 → leaf 수는 (k - 1) 증가
                    # use 개의 노드를 분할하므로 총 증가량은 use * (k-1)
                    leaves += use * (k - 1)

                    # 분할 예산 소모
                    rem -= use

                    # 다음 레벨에서의 frontier(cap)는
                    # 이번에 분할한 노드(use) 각각이 k 개의 자식을 가지므로 use * k
                    cap = use * k

                # 이 분기 타입을 처리하는 도중에 예산을 다 썼다면 전체 종료
                if rem == 0:
                    break

            # 이 순서/조합으로 얻어진 leaf 개수가 최대라면 갱신
            if leaves > best:
                best = leaves

    # dist_limit, split_limit 조건 하에서 만들 수 있는 leaf 수의 최대값 반환
    return best


In [ ]:
# ✅ 핵심 아이디어
# - 각 신호등 i: (t-1) % P_i ∈ [G_i, G_i+Y_i-1]
# - 전체를 탐색하되, “첫 번째 신호등의 노란불 순간”만 후보로 두고
#   t를 그 주기(P_0) 단위로 건너뛰며 검사 → 큰 시간 단축
# - L = lcm(P_i)까지만 탐색하면 충분 (이후는 주기 반복)
# - n ≤ 5, P_i ≤ 20 → L ≤ 55440으로 완전 탐색 가능

from math import gcd

def lcm(a, b):
    # 최소공배수: lcm(a,b) = a*b / gcd(a,b)
    return a * b // gcd(a, b)

def solution(signals):
    """
    signals: [(G1, Y1, R1), (G2, Y2, R2), ...]
      - 각 신호등 i의 주기 P_i = G_i + Y_i + R_i
      - 시각 t(1-based)에서 (t-1) % P_i 가 [G_i, G_i + Y_i - 1] 범위이면 '노란불'

    목표: 모든 신호등이 동시에 '노란불'인 가장 이른 시각 t(>=1)을 구함.
         (없으면 -1)
    """

    n = len(signals)
    periods = [sum(s) for s in signals]  # P_i = G_i + Y_i + R_i

    # 전체 패턴이 반복되는 초주기 L = lcm(P_1, ..., P_n)
    L = 1
    for p in periods:
        L = lcm(L, p)

    # --- 첫 번째 신호등(인덱스 0)을 기준으로 후보 시각을 생성 ---
    # 첫 번째 신호등이 '노란불'인 시각만 보면 충분:
    #   (t-1) % P0 ∈ [G0, G0+Y0-1] <=> t ≡ r+1 (mod P0) for some r in [G0, G0+Y0-1]
    G0, Y0, R0 = signals[0]
    P0 = periods[0]

    # 첫 번째 신호등의 "노란" 잔여시간 r에 대해 대응하는 최소 양의 시각은 t = r+1
    # 이 시각에서 P0 단위로만 증가시키면, 항상 첫 번째 신호등은 노란불 상태 유지
    yellow_points = [r + 1 for r in range(G0, G0 + Y0)]  # (Y0==0이면 빈 리스트)

    # 시각 t에서 모든 신호등이 노란불인지 검사
    def all_yellow(t):
        # 각 i에 대해 r_i = (t-1) % P_i가 [G_i, G_i + Y_i) 범위 안인지 확인
        for (G, Y, R), P in zip(signals, periods):
            r = (t - 1) % P
            if not (G <= r < G + Y):
                return False
        return True

    # 답(최소 시각). 없으면 -1 유지
    ans = -1

    # 첫 번째 신호등의 '노란' 시작 시각들(base)을 기준으로,
    # t = base, base + P0, base + 2*P0, ... (<= L) 만 검사
    #  -> 첫 번째 신호등 조건은 자동으로 만족, 나머지 신호등만 확인
    for base in yellow_points:
        # t는 [1, L] 안에서만 보면 충분 (L 이후는 패턴 반복)
        for t in range(base, L + 1, P0):
            if all_yellow(t):
                # 이 base에서 찾은 최솟값 t를 기록하고, 다음 base로 넘어감
                ans = t if ans == -1 else min(ans, t)
                break

    return ans
